In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import random
from sklearn.metrics import roc_auc_score, roc_curve
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from sklearn.preprocessing import MinMaxScaler

In [2]:
data = pd.read_csv('../data/onti_c.csv')
data.drop(['Time'], axis=1, inplace=True)
data.drop(['Crystal Position'], axis=1, inplace=True) # Beacuse has infs (try to fix later)
data.tail()

,Coolness_RHEED,R FWHM_RHEED,X FWHM_RHEED,Y FWHM_RHEED,Filtered Rate,Length,Raw Rate,Source Power
2363767,0.0,0.0,0.0,0.0,0.003712,1733.0,0.0,3.0
2363768,0.0,0.0,0.0,0.0,0.004330,1733.0,0.0,3.0
2363769,0.0,0.0,0.0,0.0,0.004020,1733.0,0.0,3.0
2363770,0.0,0.0,0.0,0.0,0.004020,1733.0,0.0,3.0
2363771,0.0,0.0,0.0,0.0,0.004640,1733.0,0.0,3.0


In [3]:
# data['Anomaly'] = data['Coolness_RHEED']
# data['Anomaly'] = (data['Anomaly'] - data['Anomaly'].rolling(50).mean()).apply(lambda x: (1 if abs(x) >= 5 else 0))

data['Anomaly'] = data['Coolness_RHEED'].rolling(50).mean()
data['Anomaly'] = ((data['Anomaly']- data['Anomaly'].shift(-435)) > 5)

temp = data[11000:len(data.index)-1000]
anom_indexes = temp[temp['Anomaly'] == 1].index
norm_indexes = temp[temp['Anomaly'] == 0].index

In [21]:
num_data = len(data.index)

train_split = 1
num_train = int(train_split * num_data)
num_test = num_data - num_train

train = data.drop('Anomaly', axis=1).values[0:num_train]
# test = data.drop('Anomaly', axis=1).values[num_train:]
# scaler = MinMaxScaler()
# train = scaler.fit_transform(train)
# test = scaler.transform(test)

num_x_signals = train.shape[1]

def get_data_entry(length=1000):
    # Allocate a new array for the batch of input-signals.
    x_shape = (length, 10000, num_x_signals)
    x_batch = np.zeros(shape=x_shape, dtype=np.float32)

    # Allocate a new array for the batch of output-signals.
    y_shape = (length, 1, 1)
    y_batch = np.zeros(shape=y_shape, dtype=np.float32)

    # Fill the batch with random sequences of data.
    for i in range(length):
        # Get a random start-index.
        # This points somewhere into the training-data.
        idx = random.choice(anom_indexes) if i % 2 == 0 else random.choice(norm_indexes)
            
        # Copy the sequences of data starting at this index.
        x_batch[i] = train[idx - 10000:idx]
        y_mean = train[idx + 385:idx + 435][0].mean()
#         y_batch[i] = [1 if (abs(train[idx + 435][0] - y_mean) >= 5) else 0]
        y_batch[i] = [[1 if i % 2 == 0 else 0, 0 if i % 2 == 0 else 1]]
        
    return (x_batch, y_batch)

def batch_generator(batch_size):
    while True:
        yield get_data_entry(length=batch_size)
        
generator = batch_generator(2)


In [22]:
def create_model():
    m = Sequential()
    m.add(GRU(units=32, input_shape=(None, num_x_signals,), return_sequences=True))
    m.add(GRU(units=16, input_shape=(None, num_x_signals,)))
    m.add(Dense(2, activation='softmax'))

    m.compile(loss='mean_squared_logarithmic_error', optimizer='adam',
              metrics=[tf.keras.metrics.AUC(name='auc', num_thresholds=10000)])
    return m

model = create_model()
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_10 (GRU)                 (None, None, 32)          4032      
_________________________________________________________________
gru_11 (GRU)                 (None, 16)                2400      
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 34        
Total params: 6,466
Trainable params: 6,466
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.fit(x=generator,
          epochs=20,
          steps_per_epoch=8,
          validation_data=get_data_entry(200))

ValueError: cannot copy sequence with size 2 to array axis with dimension 1

In [ ]:
test = get_data_entry()
# expected = test[1]
# predicted = model.predict(x=test[0], use_multiprocessing=True)

result = model.evaluate(test[0], test[1])
print("AUC: {0:.3%}".format(result[1]))

In [ ]:
model.save('new_model.h5')

In [ ]:
l_model = tf.keras.models.load_model('my_model.h5')
test = get_data_entry()
result = l_model.evaluate(test[0], test[1])
print("AUC: {0:.3%}".format(result[1]))